# ABS Quarterly National Accounts 5206 - No 2

Productivity, Unit Labour Costs, and Inflation measures from the National Accounts.

**Note:** Some charts (Solow Residual/MFP and Capital Stock comparisons) use data from the ABS Modeller's Database (1364.0.15.003), which is released one day after the Quarterly National Accounts.

## Python set-up

In [1]:
# system imports
import textwrap
from typing import Any

# analytic imports
import numpy as np
import pandas as pd
import readabs as ra
from readabs import metacol as mc
from statsmodels.tsa.filters.hp_filter import hpfilter

from mgplot import (
    calc_growth,
    line_plot_finalise,
    postcovid_plot_finalise,
    series_growth_plot_finalise,
    growth_plot_finalise,
    multi_start,
)

In [2]:
# local imports
from abs_helper import (
    get_abs_data,
    ANNUAL_CPI_TARGET_RANGE,
    QUARTERLY_CPI_TARGET,
)
from henderson import hma

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False
FILE_TYPE = "png"

# starts
bar_starts = (0, -19)
line_starts = (0, -25)

## Get data from ABS

In [3]:
abs_dict, meta, source, _ = get_abs_data("5206.0", chart_dir_suffix=" - Productivity")

# Frequently used names ...
KEY_AGGS = "5206001_Key_Aggregates"
CVM = "Chain volume measures"
CP = "Current prices"
VALUE_TEXT = "$ Millions"

In [4]:
meta = meta[~meta[mc.table].str.contains("Mock-up")]

In [5]:
# Let's see the spreadsheets we have captured
tw = textwrap.TextWrapper(width=80)
tw.wrap(", ".join(abs_dict.keys()))

['5206001_Key_Aggregates, 5206002_Expenditure_Volume_Measures,',
 '5206003_Expenditure_Current_Price, 5206004_Expenditure_Price_Indexes,',
 '5206005_Expenditure_Implicit_Price_Deflators, 5206006_Industry_GVA,',
 '5206007_Income_From_GDP, 5206008_Household_Final_Consumption_Expenditure,',
 '5206009_Changes_In_Inventories, 5206010_Agricultural_Income,',
 '5206011_National_Income_Account, 5206012_National_Capital_Account,',
 '5206013_NFC_Income, 5206014_PriNFC_Income, 5206015_PubNFC_Income,',
 '5206016_FC_Income, 5206017_Gen_Govt_Income_Account,',
 '5206018_Nat_Gen_Govt_Income_Account, 5206019_StateLocal_Gen_Govt_Income_Account,',
 '5206020_Household_Income, 5206021_External_Account, 5206022_Taxes,',
 '5206023_Social_Assistance_Benefits, 5206024_Selected_Analytical_Series,',
 '5206025_SFD_Summary, 5206026_SFD_NSW, 5206027_SFD_VIC, 5206028_SFD_QLD,',
 '5206029_SFD_SA, 5206030_SFD_WA, 5206031_SFD_TAS, 5206032_SFD_NT,',
 '5206033_SFD_ACT, 5206034_Key_Aggregates_and_Analytical_Series_Annual_d

## Plot

### Implicit price deflators

In [6]:
def price_deflators() -> dict:
    """Calculate and plot the implicit price deflators for key series."""

    table = "5206005_Expenditure_Implicit_Price_Deflators"
    data = abs_dict[table]

    keys = {
        "GDP": "GROSS DOMESTIC PRODUCT ;",
        "GNE": "Gross national expenditure ;",
        "Households": "Households ;  Final consumption expenditure ;",
    }

    common: dict[str, Any] = {
        "rfooter": f"{source}",
        "lfooter": "Australia. Seasonally adjusted. ",
        "legend": {"loc": "best", "fontsize": "xx-small"},
        "pre_tag": "deflators-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }

    deflators = {}  # used in calculations below ...
    for prefix, key in keys.items():
        ident = meta[(meta[mc.did] == key) & (meta[mc.table] == table)][mc.id].iloc[0]
        series = data[ident]
        deflators[prefix] = series / series.iloc[-1]  # rebase
        title = f"Growth in {prefix} Implicit Price Deflator"

        growth = calc_growth(series)
        series_growth_plot_finalise(
            series,
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            axhline=QUARTERLY_CPI_TARGET,
            title=title,
            plot_from=-19,
            tag="deflators-growth",
            zero_y=True,
            **common,
        )

        growth[growth.columns[0]].name = title
        line_plot_finalise(
            growth[growth.columns[0]],
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            title=f"Annual {title}",
            ylabel="Per cent growth",
            tag="deflators-growth-annual",
            annotate=True,
            y0=True,
            **common,
        )

    return deflators


DEFLATORS = price_deflators()

### Unit Labour Cost growth (calculated)

**Calculated ULC** = Compensation of Employees (COE) / GDP (Chain Volume Measures)

This simple calculation diverges from the official ABS ULC index (Table 42). According to the [ABS methodology](https://www.abs.gov.au/statistics/detailed-methodology-information/concepts-sources-methods/australian-system-national-accounts-concepts-sources-and-methods/2020-21/chapter-20-analytical-measures/unit-labour-costs), the official formula is:

**Official ABS ULC** = (Labour Costs / Hours worked by employees) / (GDP / Total hours worked)

Where **Labour Costs = COE + payroll tax - employment subsidies**

The key differences are:

1. **Employment subsidies**: The ABS subtracts all employment subsidies from labour costs - not just COVID-era JobKeeper, but ongoing programs like apprentice wage subsidies, disability employment subsidies, and other workforce programs. This makes LC < COE.

2. **Hours worked adjustment**: The official formula uses employee hours in the numerator but total hours (including self-employed) in the denominator. If the employee share of total hours has risen over time, this scaling factor falls, causing official ULC to grow slower.

3. **Payroll tax**: Added to labour costs, but this is small relative to subsidies.

**Interpretation**: The official ULC measures what employers actually pay out of pocket per unit of output. Our simple COE/GDP measures what workers receive relative to output. For understanding household income and spending power, the COE-based measure may be more relevant. For understanding employer cost competitiveness, the official measure is appropriate.

The comparison charts below show both the cumulative index divergence (~12% by 2024) and the quarter-by-quarter growth rate differences.

In [7]:
def get_ulc() -> None:
    """Calculated unit labour costs: COE / GDP(CVM)."""

    series_type = "Seasonally Adjusted"

    # GDP Chain Volume Measures (from Key Aggregates)
    gdp_selector = {
        KEY_AGGS: mc.table,
        series_type: mc.stype,
        "Gross domestic product: Chain volume measures ;": mc.did,
        "$": mc.unit,
    }
    _table, gdp_id, _units = ra.find_abs_id(meta, gdp_selector, verbose=False)
    gdp = abs_dict[KEY_AGGS][gdp_id].dropna()

    # Compensation of Employees (from Income from GDP)
    income_table = "5206007_Income_From_GDP"
    coe_selector = {
        income_table: mc.table,
        series_type: mc.stype,
        "Compensation of employees ;": mc.did,
    }
    _table, coe_id, _units = ra.find_abs_id(meta, coe_selector, exact=True, verbose=False)
    coe = abs_dict[income_table][coe_id].dropna()

    # Calculate ULC = COE / GDP(CVM) - pandas handles alignment
    ulc = (coe / gdp).dropna()

    common = {
        "rfooter": f"{source}",
        "lfooter": f"Australia. {series_type}. ULC = COE / GDP(CVM). ",
        "pre_tag": "ulc-calc-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }

    # --- Nominal ULC level charts ---
    ulc.name = "Calculated Nominal ULC"

    multi_start(
        ulc,
        function=line_plot_finalise,
        starts=line_starts,
        title="Calculated Nominal Unit Labour Costs",
        ylabel="Ratio (COE / GDP CVM)",
        annotate=True,
        **common,
    )

    # --- Growth charts ---
    growth = calc_growth(ulc)
    growth_plot_finalise(
        growth,
        title="Calculated Unit Labour Costs Growth",
        ylabel="Per cent growth",
        plot_from=-19,
        tag="growth",
        **common,
    )

    line_plot_finalise(
        growth[growth.columns[0]],
        title="Calculated Unit Labour Costs Growth (YoY)",
        ylabel="Per cent growth",
        y0=True,
        tag="growth-annual",
        **common,
    )

    # --- Compare with official ULC index ---
    ulc_table = "5206042_Unit_Labour_Costs"
    table42_meta = meta[meta[mc.table] == ulc_table]
    table42_sa = table42_meta[table42_meta[mc.stype] == series_type]
    
    # Find the official nominal ULC (whole economy, not non-farm)
    official_rows = table42_sa[
        (table42_sa[mc.did].str.contains("Nominal", case=False, na=False)) &
        (table42_sa[mc.did].str.contains("unit labour cost", case=False, na=False)) &
        (~table42_sa[mc.did].str.contains("Non-farm", case=False, na=False)) &
        (~table42_sa[mc.did].str.contains("Percentage", case=False, na=False))
    ]
    
    if len(official_rows) == 0:
        print("Could not find official Nominal ULC series")
        return
    
    official_row = official_rows.iloc[0]
    official_id = official_row[mc.id]
    official_ulc = abs_dict[ulc_table][official_id].dropna()
    
    # Rebase both to start = 100 for comparison
    ulc_rebased = ulc / ulc.iloc[0] * 100
    official_rebased = official_ulc / official_ulc.iloc[0] * 100
    
    compare_df = pd.DataFrame({
        "Calculated (COE/GDP)": ulc_rebased,
        "Official (Table 42)": official_rebased,
    }).dropna()
    
    line_plot_finalise(
        compare_df,
        title="ULC Comparison: Calculated vs Official Index",
        ylabel="Index (start = 100)",
        annotate=True,
        legend={"loc": "best", "fontsize": 9},
        tag="vs-official",
        lfooter=f"Australia. {series_type}. Calculated = COE/GDP(CVM). Official = Table 42 Nominal ULC. ",
        rfooter=f"{source}",
        pre_tag="ulc-calc-",
        show=SHOW,
        file_type=FILE_TYPE,
    )
    
    # Growth comparison
    calc_growth_yoy = ulc.pct_change(periods=4) * 100
    official_growth_yoy = official_ulc.pct_change(periods=4) * 100
    
    growth_compare = pd.DataFrame({
        "Calculated (COE/GDP)": calc_growth_yoy,
        "Official (Table 42)": official_growth_yoy,
    }).dropna()
    
    line_plot_finalise(
        growth_compare,
        title="ULC Growth Comparison: Calculated vs Official",
        ylabel="Per cent (YoY)",
        annotate=True,
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth-vs-official",
        lfooter=f"Australia. {series_type}. ",
        rfooter=f"{source}",
        pre_tag="ulc-calc-",
        show=SHOW,
        file_type=FILE_TYPE,
    )


get_ulc()

### Official Unit Labour Costs (Table 42)

In [8]:
def official_ulc() -> None:
    """Plot official Unit Labour Costs from ABS Table 42."""

    table = "5206042_Unit_Labour_Costs"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"

    # Get all ULC series from Table 42
    ulc_series = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
    ]

    common: dict[str, Any] = {
        "rfooter": f"{source} {table}",
        "lfooter": f"Australia. {series_type}. ",
        "pre_tag": "ulc-official-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }

    for _index, row in ulc_series.iterrows():
        series_id, units, did = row[mc.id], row[mc.unit], row[mc.did]
        series = data[series_id].dropna()
        
        title = did.replace(" ;", "").strip()
        
        # Index charts
        multi_start(
            series,
            function=line_plot_finalise,
            starts=line_starts,
            title=title,
            ylabel=units,
            annotate=True,
            **common,
        )

        # Growth charts
        series_growth_plot_finalise(
            series,
            title=f"{title} growth",
            plot_from=-19,
            tag="growth",
            **common,
        )


official_ulc()

### Hourly Compensation of Employees

In [9]:
def hourly_coe() -> None:
    """Plot Hourly Compensation of Employees (COE) from Selected Analytical Series."""

    table = "5206024_Selected_Analytical_Series"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"

    # Hourly COE series
    coe_dids = [
        "Compensation of employees per hour: Current prices ;",
        "Non-farm compensation of employees per hour: Current prices ;",
    ]

    common: dict[str, Any] = {
        "rfooter": f"{source} {table}",
        "lfooter": f"Australia. {series_type}. Current prices. ",
        "pre_tag": "hourly-coe-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }

    for did in coe_dids:
        row = meta[
            (meta[mc.table] == table)
            & (meta[mc.stype] == series_type)
            & (meta[mc.did] == did)
        ].iloc[0]
        series_id, units = row[mc.id], row[mc.unit]
        series = data[series_id].dropna()
        
        title = did.replace(": Current prices ;", "").strip()

        # Level charts
        multi_start(
            series,
            function=line_plot_finalise,
            starts=line_starts,
            title=title,
            ylabel=units,
            annotate=True,
            **common,
        )

        # COVID recovery
        postcovid_plot_finalise(
            series,
            title=title,
            ylabel=units,
            tag="covid",
            annotate=[False, True],
            **common,
        )

        # Growth charts
        series_growth_plot_finalise(
            series,
            title=f"{title} growth",
            plot_from=-19,
            tag="growth",
            **common,
        )

    # Comparison chart: All sectors vs Non-farm
    all_row = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.did] == coe_dids[0])
    ].iloc[0]
    nonfarm_row = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.did] == coe_dids[1])
    ].iloc[0]
    
    comparison = pd.DataFrame({
        "All sectors": data[all_row[mc.id]],
        "Non-farm": data[nonfarm_row[mc.id]],
    }).dropna()
    
    line_plot_finalise(
        comparison,
        title="Hourly COE: All sectors vs Non-farm",
        ylabel=all_row[mc.unit],
        annotate=True,
        **common,
    )
    
    # Annual growth comparison
    growth_comparison = comparison.pct_change(periods=4) * 100
    line_plot_finalise(
        growth_comparison.dropna(),
        title="Hourly COE Growth (YoY): All sectors vs Non-farm",
        ylabel="Per cent",
        tag="growth-comparison",
        annotate=True,
        y0=True,
        **common,
    )


hourly_coe()

### Labour Productivity

In [10]:
def labour_productivity() -> None:
    """Plot labour productivity measures with comparisons and long-run trends."""

    table = KEY_AGGS
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"

    # Productivity series from Key Aggregates
    productivity_dids = {
        "GDP per hour worked": "GDP per hour worked: Index ;",
        "GVA per hour worked (Market sector)": "Gross value added per hour worked market sector: Index ;",
    }

    common: dict[str, Any] = {
        "rfooter": f"{source} {table}",
        "lfooter": f"Australia. {series_type}. ",
        "pre_tag": "productivity-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }

    # --- Individual productivity charts ---
    productivity_data = {}
    for label, did in productivity_dids.items():
        selector = {
            table: mc.table,
            series_type: mc.stype,
            did: mc.did,
        }
        try:
            _table, series_id, units = ra.find_abs_id(meta, selector, verbose=False)
            series = data[series_id].dropna()
            productivity_data[label] = series

            # Level charts
            multi_start(
                series,
                function=line_plot_finalise,
                starts=line_starts,
                title=label,
                ylabel=units,
                annotate=True,
                **common,
            )

            # COVID recovery
            postcovid_plot_finalise(
                series,
                title=label,
                ylabel=units,
                tag="covid",
                annotate=[False, True],
                **common,
            )

            # Growth charts
            series_growth_plot_finalise(
                series,
                title=f"{label} growth",
                plot_from=-19,
                tag="growth",
                **common,
            )
        except Exception as e:
            print(f"Could not find {label}: {e}")

    if not productivity_data:
        print("No productivity series found")
        return

    # --- Productivity comparison chart ---
    comparison = pd.DataFrame(productivity_data).dropna()
    # Rebase to common start
    comparison = comparison / comparison.iloc[0] * 100
    line_plot_finalise(
        comparison,
        title="Labour Productivity: GDP vs Market Sector GVA",
        ylabel="Index (start = 100)",
        annotate=True,
        legend={"loc": "best", "fontsize": 9},
        **common,
    )

    # Annual growth comparison
    growth_comparison = pd.DataFrame(productivity_data).pct_change(periods=4) * 100
    line_plot_finalise(
        growth_comparison.dropna(),
        title="Labour Productivity Growth (YoY)",
        ylabel="Per cent",
        tag="growth-comparison",
        annotate=True,
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        **common,
    )

    # --- Long-run productivity trends with Henderson MA ---
    for label, series in productivity_data.items():
        growth = series.pct_change(periods=4) * 100
        growth = growth.dropna()
        
        hma_term = 13
        smoothed = hma(growth, hma_term)
        
        df = pd.DataFrame({
            "Annual growth": growth,
            f"{hma_term}-term HMA": smoothed,
        })
        
        line_plot_finalise(
            df,
            title=f"{label} - Long-run Growth Trends",
            ylabel="Per cent growth (YoY)",
            width=[1, 3],
            annotate=[False, True],
            y0=True,
            legend={"loc": "best", "fontsize": 9},
            tag="long-run",
            **common,
        )

    # --- Real Unit Labour Costs (inverse relationship with productivity) ---
    # Find real ULC series directly from meta
    real_ulc_rows = meta[
        (meta[mc.table] == table) &
        (meta[mc.stype] == series_type) &
        (meta[mc.did].str.contains("Real unit labour costs", case=False, na=False))
    ]
    
    ulc_data = {}
    for _, row in real_ulc_rows.iterrows():
        series_id = row[mc.id]
        units = row[mc.unit]
        did = row[mc.did]
        label = did.replace(" ;", "").replace(": Index", "").strip()
        
        series = data[series_id].dropna()
        ulc_data[label] = series
        
        multi_start(
            series,
            function=line_plot_finalise,
            starts=line_starts,
            title=label,
            ylabel=units,
            annotate=True,
            **common,
        )

    # --- Productivity vs Real ULC comparison ---
    if ulc_data and productivity_data:
        # Use GDP productivity and total real ULC
        gdp_prod = productivity_data.get("GDP per hour worked")
        # Find the basic real ULC (not non-farm)
        real_ulc = None
        for key, val in ulc_data.items():
            if "Non-farm" not in key and "non-farm" not in key:
                real_ulc = val
                break
        
        if gdp_prod is not None and real_ulc is not None:
            # Rebase both to 100
            comp_df = pd.DataFrame({
                "Labour Productivity (GDP/hour)": gdp_prod / gdp_prod.iloc[0] * 100,
                "Real Unit Labour Costs": real_ulc / real_ulc.iloc[0] * 100,
            }).dropna()
            
            line_plot_finalise(
                comp_df,
                title="Labour Productivity vs Real Unit Labour Costs",
                ylabel="Index (start = 100)",
                annotate=True,
                legend={"loc": "best", "fontsize": 9},
                tag="vs-ulc",
                **common,
            )


labour_productivity()

### Solow Residual (Multifactor Productivity)

The Solow residual measures output growth not explained by growth in labour and capital inputs:

$MFP = \frac{Y}{L^\alpha \cdot K^{1-\alpha}}$

Where:
- Y = Real GDP (chain volume measures)
- L = Hours worked
- K = Capital stock (estimated via perpetual inventory from GFCF)
- α = Labour share of income (from COE / Total Factor Income)

In [11]:
def solow_residual() -> None:
    """Calculate and plot the Solow residual (Multifactor Productivity).
    
    MFP = Y / (L^α * K^(1-α))
    
    Uses:
    - GDP (CVM) for output
    - Hours worked for labour input
    - Capital stock from ABS Modellers Database (1364.0.15.003)
    - Labour share from COE / Total Factor Income
    """

    series_type = "Seasonally Adjusted"
    
    # --- Get GDP (Chain Volume Measures) ---
    gdp_table = KEY_AGGS
    selector = {
        gdp_table: mc.table,
        series_type: mc.stype,
        "Gross domestic product: Chain volume measures ;": mc.did,
        "$": mc.unit,
    }
    _table, gdp_id, _units = ra.find_abs_id(meta, selector, verbose=False)
    gdp = abs_dict[gdp_table][gdp_id].dropna()
    
    # --- Get Hours Worked (from Key Aggregates) ---
    hours_table = KEY_AGGS
    selector = {
        hours_table: mc.table,
        series_type: mc.stype,
        "Hours worked: Index ;": mc.did,
    }
    _table, hours_id, _units = ra.find_abs_id(meta, selector, verbose=False)
    hours = abs_dict[hours_table][hours_id].dropna()
    
    # --- Get Labour Share (COE / Total Factor Income) ---
    income_table = "5206007_Income_From_GDP"
    
    # Compensation of employees
    selector = {
        income_table: mc.table,
        series_type: mc.stype,
        "Compensation of employees ;": mc.did,
    }
    _table, coe_id, _units = ra.find_abs_id(meta, selector, exact=True, verbose=False)
    coe = abs_dict[income_table][coe_id].dropna()
    
    # Total factor income
    selector = {
        income_table: mc.table,
        series_type: mc.stype,
        "Total factor income ;": mc.did,
    }
    _table, tfi_id, _units = ra.find_abs_id(meta, selector, exact=True, verbose=False)
    tfi = abs_dict[income_table][tfi_id].dropna()
    
    # Labour share (alpha) - pandas handles alignment
    labour_share = (coe / tfi).dropna()
    
    # --- Get Capital Stock from ABS Modellers Database (1364.0.15.003) ---
    print("Fetching capital stock from ABS Modellers Database (1364.0.15.003)...")
    md_dict, md_meta = ra.read_abs_cat("1364.0.15.003")
    
    capital_did = "Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;"
    capital_rows = md_meta[
        (md_meta[mc.did] == capital_did) &
        (md_meta[mc.stype] == "Seasonally Adjusted")
    ]
    
    if len(capital_rows) == 0:
        # Try partial match
        capital_rows = md_meta[
            md_meta[mc.did].str.contains("Net capital stock", case=False, na=False) &
            md_meta[mc.did].str.contains("Chain volume", case=False, na=False) &
            (md_meta[mc.stype] == "Seasonally Adjusted")
        ]
    
    if len(capital_rows) == 0:
        print("Could not find capital stock series")
        return
        
    capital_row = capital_rows.iloc[0]
    capital_id = capital_row[mc.id]
    capital_table = capital_row[mc.table]
    capital = md_dict[capital_table][capital_id].dropna()
    print(f"Using: {capital_row[mc.did]}")
    
    # Use smoothed alpha (long-run average)
    alpha_smooth = labour_share.mean()
    
    # --- Normalize inputs to index form ---
    L_norm = hours / hours.iloc[0]
    K_norm = capital / capital.iloc[0]
    Y_norm = gdp / gdp.iloc[0]
    
    # --- Calculate Solow Residual (MFP) - pandas handles alignment ---
    # MFP = Y / (L^α * K^(1-α))
    mfp = (Y_norm / (L_norm ** alpha_smooth * K_norm ** (1 - alpha_smooth)) * 100).dropna()
    
    # --- Calculate MFP growth ---
    mfp_growth = mfp.pct_change(periods=4) * 100  # Annual growth
    mfp_growth = mfp_growth.dropna()
    
    # --- Apply smoothing filters ---
    # HP filter (lambda = 1600 for quarterly data)
    mfp_cycle, mfp_trend_hp = hpfilter(mfp, lamb=1600)
    mfp_trend_hp = pd.Series(mfp_trend_hp, index=mfp.index)
    
    # 13-term Henderson moving average
    mfp_trend_hma = hma(mfp, 13)
    
    # HP filter on growth
    growth_cycle, growth_trend_hp = hpfilter(mfp_growth, lamb=1600)
    growth_trend_hp = pd.Series(growth_trend_hp, index=mfp_growth.index)
    
    # 13-term Henderson on growth
    growth_trend_hma = hma(mfp_growth, 13)
    
    # --- Plotting ---
    common: dict[str, Any] = {
        "rfooter": f"{source} + ABS 1364.0.15.003",
        "lfooter": f"Australia. {series_type}. Calculated Solow residual. ",
        "pre_tag": "mfp-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }
    
    # Plot MFP level with trends
    mfp_df = pd.DataFrame({
        "MFP (Solow Residual)": mfp,
        "HP Filter Trend": mfp_trend_hp,
        "13-term Henderson MA": mfp_trend_hma,
    }).dropna()
    
    line_plot_finalise(
        mfp_df,
        title="Multifactor Productivity (Solow Residual)",
        ylabel="Index",
        width=[1, 2, 2],
        annotate=[False, False, True],
        legend={"loc": "best", "fontsize": 9},
        **common,
    )
    
    # Plot just the trends
    trends_df = pd.DataFrame({
        "HP Filter Trend": mfp_trend_hp,
        "13-term Henderson MA": mfp_trend_hma,
    }).dropna()
    
    line_plot_finalise(
        trends_df,
        title="MFP Trends: HP Filter vs Henderson MA",
        ylabel="Index",
        width=[2, 2],
        annotate=True,
        legend={"loc": "best", "fontsize": 9},
        tag="trends",
        **common,
    )
    
    # Plot MFP growth with trends
    growth_df = pd.DataFrame({
        "MFP Growth (YoY)": mfp_growth,
        "HP Filter Trend": growth_trend_hp,
        "13-term Henderson MA": growth_trend_hma,
    }).dropna()
    
    line_plot_finalise(
        growth_df,
        title="MFP Growth with Smoothed Trends",
        ylabel="Per cent (YoY)",
        width=[1, 2, 2],
        annotate=[False, False, True],
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth",
        **common,
    )
    
    # Plot comparison: Labour productivity vs MFP
    lp_selector = {
        KEY_AGGS: mc.table,
        series_type: mc.stype,
        "GDP per hour worked: Index ;": mc.did,
    }
    _table, lp_id, _units = ra.find_abs_id(meta, lp_selector, verbose=False)
    labour_prod = abs_dict[KEY_AGGS][lp_id].dropna()
    
    comparison_df = pd.DataFrame({
        "Labour Productivity": labour_prod,
        "MFP (Solow Residual)": mfp,
    }).dropna()
    # Rebase to 100 at the start of the visible data
    comparison_df = comparison_df / comparison_df.iloc[0] * 100
    
    line_plot_finalise(
        comparison_df,
        title="Labour Productivity vs Multifactor Productivity",
        ylabel="Index (start = 100)",
        annotate=True,
        legend={"loc": "best", "fontsize": 9},
        tag="vs-lp",
        **common,
    )
    
    # Print summary statistics
    print(f"Labour share (α): {alpha_smooth:.3f}")
    print(f"MFP level (latest): {mfp.iloc[-1]:.1f}")
    print(f"MFP growth (latest YoY): {mfp_growth.iloc[-1]:.2f}%")
    print(f"MFP trend growth (HP, latest): {growth_trend_hp.iloc[-1]:.2f}%")
    print(f"MFP trend growth (HMA, latest): {growth_trend_hma.iloc[-1]:.2f}%")


solow_residual()

Fetching capital stock from ABS Modellers Database (1364.0.15.003)...
Using: Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;
Labour share (α): 0.545
MFP level (latest): 142.3
MFP growth (latest YoY): 0.34%
MFP trend growth (HP, latest): -0.72%
MFP trend growth (HMA, latest): -0.25%


### Capital Stock: Perpetual Inventory vs ABS Modellers Database

In [12]:
def compare_capital_stock() -> None:
    """Compare perpetual inventory capital estimate with ABS Modellers Database."""

    series_type = "Seasonally Adjusted"
    
    # --- Calculate perpetual inventory capital (same as in solow_residual) ---
    gfcf_table = "5206002_Expenditure_Volume_Measures"
    selector = {
        gfcf_table: mc.table,
        series_type: mc.stype,
        "All sectors ;  Gross fixed capital formation ;": mc.did,
    }
    _table, gfcf_id, _units = ra.find_abs_id(meta, selector, verbose=False)
    gfcf = abs_dict[gfcf_table][gfcf_id].dropna()
    
    # Perpetual inventory method
    depreciation_rate = 0.025 / 4  # ~2.5% annual = 0.625% quarterly
    capital_pi = pd.Series(index=gfcf.index, dtype=float)
    capital_pi.iloc[0] = gfcf.iloc[0] * 40  # Initial estimate
    for i in range(1, len(gfcf)):
        capital_pi.iloc[i] = (1 - depreciation_rate) * capital_pi.iloc[i-1] + gfcf.iloc[i]
    
    # --- Fetch official capital stock from ABS Modellers Database (1364.0.15.003) ---
    print("Fetching ABS Modellers Database (1364.0.15.003)...")
    md_dict, md_meta = ra.read_abs_cat("1364.0.15.003")
    
    # Find the capital stock series
    capital_did = "Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;"
    capital_rows = md_meta[
        (md_meta[mc.did] == capital_did) &
        (md_meta[mc.stype] == "Seasonally Adjusted")
    ]
    
    if len(capital_rows) == 0:
        # Try partial match
        capital_rows = md_meta[
            md_meta[mc.did].str.contains("Net capital stock", case=False, na=False) &
            md_meta[mc.did].str.contains("Chain volume", case=False, na=False) &
            (md_meta[mc.stype] == "Seasonally Adjusted")
        ]
    
    if len(capital_rows) == 0:
        print("Could not find capital stock series. Available series:")
        print(md_meta[mc.did].unique()[:20])
        return
    
    capital_row = capital_rows.iloc[0]
    capital_id = capital_row[mc.id]
    capital_table = capital_row[mc.table]
    capital_units = capital_row[mc.unit]
    
    capital_abs = md_dict[capital_table][capital_id].dropna()
    print(f"Found: {capital_row[mc.did]}")
    print(f"Units: {capital_units}")
    print(f"Period: {capital_abs.index[0]} to {capital_abs.index[-1]}")
    
    common: dict[str, Any] = {
        "rfooter": f"{source} + ABS 1364.0.15.003",
        "lfooter": f"Australia. {series_type}. PI assumes 2.5% p.a. depreciation. ",
        "pre_tag": "capital-comparison-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }
    
    # --- Plot levels ---
    comparison_df = pd.DataFrame({
        "Perpetual Inventory (from GFCF)": capital_pi,
        "ABS Modellers Database": capital_abs,
    }).dropna()
    comparison_df, units = ra.recalibrate(comparison_df, capital_units)
    
    line_plot_finalise(
        comparison_df,
        title="Capital Stock: Perpetual Inventory vs ABS Official",
        ylabel=units,
        annotate=True,
        legend={"loc": "best", "fontsize": 9},
        **common,
    )
    
    # --- Plot growth rates ---
    pi_growth = capital_pi.pct_change(periods=4) * 100
    abs_growth = capital_abs.pct_change(periods=4) * 100
    
    growth_df = pd.DataFrame({
        "Perpetual Inventory": pi_growth,
        "ABS Modellers Database": abs_growth,
    }).dropna()
    
    line_plot_finalise(
        growth_df,
        title="Capital Stock Growth (YoY): PI vs ABS Official",
        ylabel="Per cent",
        annotate=True,
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth",
        **common,
    )
    
    # --- Plot rebased index comparison ---
    pi_rebased = capital_pi / capital_pi.iloc[0] * 100
    abs_rebased = capital_abs / capital_abs.iloc[0] * 100
    
    index_df = pd.DataFrame({
        "Perpetual Inventory": pi_rebased,
        "ABS Modellers Database": abs_rebased,
    }).dropna()
    
    line_plot_finalise(
        index_df,
        title="Capital Stock Index: PI vs ABS Official",
        ylabel="Index (start = 100)",
        annotate=True,
        legend={"loc": "best", "fontsize": 9},
        tag="index",
        **common,
    )
    
    # --- Summary statistics ---
    correlation = comparison_df.iloc[:, 0].corr(comparison_df.iloc[:, 1])
    growth_corr = growth_df.iloc[:, 0].corr(growth_df.iloc[:, 1])
    print(f"\nCorrelation (levels): {correlation:.4f}")
    print(f"Correlation (growth): {growth_corr:.4f}")
    print(f"PI latest: {comparison_df.iloc[-1, 0]:.1f} {units}")
    print(f"ABS latest: {comparison_df.iloc[-1, 1]:.1f} {units}")
    print(f"Ratio (PI/ABS): {comparison_df.iloc[-1, 0] / comparison_df.iloc[-1, 1]:.3f}")


compare_capital_stock()

Fetching ABS Modellers Database (1364.0.15.003)...
Found: Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;
Units: $ Millions
Period: 1959Q3 to 2025Q3

Correlation (levels): 0.9972
Correlation (growth): 0.9671
PI latest: 12.1 $ Trillions
ABS latest: 3.9 $ Trillions
Ratio (PI/ABS): 3.087


### Capital Stock Growth and Capital Deepening

Capital deepening measures capital per hour worked (K/L), showing how much capital each unit of labour has available. Rising capital deepening typically supports labour productivity growth.

In [13]:
def capital_stock_and_deepening() -> None:
    """Plot capital stock growth and capital deepening (K/L)."""

    series_type = "Seasonally Adjusted"
    
    # --- Fetch capital stock from ABS Modellers Database (1364.0.15.003) ---
    print("Fetching capital stock from ABS Modellers Database (1364.0.15.003)...")
    md_dict, md_meta = ra.read_abs_cat("1364.0.15.003")
    
    capital_did = "Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;"
    capital_rows = md_meta[
        (md_meta[mc.did] == capital_did) &
        (md_meta[mc.stype] == "Seasonally Adjusted")
    ]
    
    if len(capital_rows) == 0:
        capital_rows = md_meta[
            md_meta[mc.did].str.contains("Net capital stock", case=False, na=False) &
            md_meta[mc.did].str.contains("Chain volume", case=False, na=False) &
            (md_meta[mc.stype] == "Seasonally Adjusted")
        ]
    
    if len(capital_rows) == 0:
        print("Could not find capital stock series")
        return
    
    capital_row = capital_rows.iloc[0]
    capital_id = capital_row[mc.id]
    capital_table = capital_row[mc.table]
    capital = md_dict[capital_table][capital_id].dropna()
    print(f"Using: {capital_row[mc.did]}")
    
    # --- Get Hours Worked (from Key Aggregates) ---
    hours_selector = {
        KEY_AGGS: mc.table,
        series_type: mc.stype,
        "Hours worked: Index ;": mc.did,
    }
    _table, hours_id, _units = ra.find_abs_id(meta, hours_selector, verbose=False)
    hours = abs_dict[KEY_AGGS][hours_id].dropna()
    
    common: dict[str, Any] = {
        "rfooter": f"ABS 1364.0.15.003 (Modeller's Database)",
        "lfooter": f"Australia. {series_type}. ",
        "pre_tag": "capital-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }
    
    # --- Capital Stock Growth ---
    capital_growth = capital.pct_change(periods=4) * 100
    capital_growth = capital_growth.dropna()
    capital_growth.name = "Capital Stock Growth"
    
    # Apply Henderson smoothing
    hma_term = 13
    capital_growth_hma = hma(capital_growth, hma_term)
    
    growth_df = pd.DataFrame({
        "Annual growth": capital_growth,
        f"{hma_term}-term HMA": capital_growth_hma,
    })
    
    line_plot_finalise(
        growth_df,
        title="Capital Stock Growth (Net, Chain Volume Measures)",
        ylabel="Per cent (YoY)",
        width=[1, 3],
        annotate=[False, True],
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth",
        **common,
    )
    
    # Recent history
    line_plot_finalise(
        growth_df.iloc[-60:],
        title="Capital Stock Growth (Net, Chain Volume Measures)",
        ylabel="Per cent (YoY)",
        width=[1, 3],
        annotate=[False, True],
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth-recent",
        **common,
    )
    
    # --- Capital Deepening (K/L) ---
    # Normalize both to index form
    K_norm = capital / capital.iloc[0] * 100
    L_norm = hours / hours.iloc[0] * 100
    
    # Capital deepening = K/L (indexed) - pandas handles alignment
    capital_deepening = (K_norm / L_norm * 100).dropna()
    capital_deepening.name = "Capital Deepening (K/L)"
    
    common_kl: dict[str, Any] = {
        "rfooter": f"ABS 1364.0.15.003 + {source}",
        "lfooter": f"Australia. {series_type}. Capital per hour worked (indexed). ",
        "pre_tag": "capital-deepening-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }
    
    # Level chart
    multi_start(
        capital_deepening,
        function=line_plot_finalise,
        starts=line_starts,
        title="Capital Deepening (Capital per Hour Worked)",
        ylabel=f"Index ({capital_deepening.index[0]} = 100)",
        annotate=True,
        **common_kl,
    )
    
    # Growth chart
    kl_growth = capital_deepening.pct_change(periods=4) * 100
    kl_growth = kl_growth.dropna()
    kl_growth_hma = hma(kl_growth, hma_term)
    
    kl_growth_df = pd.DataFrame({
        "Annual growth": kl_growth,
        f"{hma_term}-term HMA": kl_growth_hma,
    })
    
    line_plot_finalise(
        kl_growth_df,
        title="Capital Deepening Growth",
        ylabel="Per cent (YoY)",
        width=[1, 3],
        annotate=[False, True],
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth",
        **common_kl,
    )
    
    # --- Comparison: Capital Deepening vs Labour Productivity ---
    lp_selector = {
        KEY_AGGS: mc.table,
        series_type: mc.stype,
        "GDP per hour worked: Index ;": mc.did,
    }
    _table, lp_id, _units = ra.find_abs_id(meta, lp_selector, verbose=False)
    labour_prod = abs_dict[KEY_AGGS][lp_id].dropna()
    labour_prod_norm = labour_prod / labour_prod.iloc[0] * 100
    
    comparison_df = pd.DataFrame({
        "Capital Deepening (K/L)": capital_deepening,
        "Labour Productivity (Y/L)": labour_prod_norm,
    }).dropna()
    
    line_plot_finalise(
        comparison_df,
        title="Capital Deepening vs Labour Productivity",
        ylabel="Index (start = 100)",
        annotate=True,
        legend={"loc": "best", "fontsize": 9},
        tag="vs-productivity",
        **common_kl,
    )
    
    # Growth comparison
    lp_growth = labour_prod_norm.pct_change(periods=4) * 100
    growth_compare = pd.DataFrame({
        "Capital Deepening": kl_growth,
        "Labour Productivity": lp_growth,
    }).dropna()
    
    line_plot_finalise(
        growth_compare,
        title="Capital Deepening vs Labour Productivity Growth",
        ylabel="Per cent (YoY)",
        annotate=True,
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth-vs-productivity",
        **common_kl,
    )
    
    print(f"Capital deepening (latest): {capital_deepening.iloc[-1]:.1f}")
    print(f"Capital deepening growth (latest YoY): {kl_growth.iloc[-1]:.2f}%")


capital_stock_and_deepening()

Fetching capital stock from ABS Modellers Database (1364.0.15.003)...
Using: Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;
Capital deepening (latest): 922.6
Capital deepening growth (latest YoY): 0.93%


### Capital Productivity

Capital productivity measures **output per unit of capital (Y/K)** — essentially how efficiently capital is being used to produce output. The inverse (K/Y) is the capital-output ratio, which shows how much capital is needed to produce one unit of output.

**Relationship between productivity measures:**

| Measure | Formula | Interpretation |
|---------|---------|----------------|
| Labour productivity | Y/L | Output per hour worked |
| Capital productivity | Y/K | Output per unit of capital |
| Capital deepening | K/L | Capital per hour worked |
| MFP (Solow residual) | Y / (L^α × K^(1-α)) | Efficiency of combined inputs |

These measures are related by the identity:

**Labour productivity = Capital deepening × Capital productivity**

Or: `Y/L = (K/L) × (Y/K)`

Capital productivity often **declines** during periods of heavy investment (capital deepening) as new capital takes time to become fully productive. One of Kaldor's "stylized facts" of growth is that the capital-output ratio tends to be relatively stable over the long run, implying stable capital productivity in trend terms.

In [14]:
def capital_productivity() -> None:
    """Plot capital productivity (Y/K) and capital-output ratio (K/Y)."""

    series_type = "Seasonally Adjusted"
    
    # --- Get GDP (Chain Volume Measures) ---
    gdp_selector = {
        KEY_AGGS: mc.table,
        series_type: mc.stype,
        "Gross domestic product: Chain volume measures ;": mc.did,
        "$": mc.unit,
    }
    _table, gdp_id, _units = ra.find_abs_id(meta, gdp_selector, verbose=False)
    gdp = abs_dict[KEY_AGGS][gdp_id].dropna()
    
    # --- Fetch capital stock from ABS Modellers Database (1364.0.15.003) ---
    print("Fetching capital stock from ABS Modellers Database (1364.0.15.003)...")
    md_dict, md_meta = ra.read_abs_cat("1364.0.15.003")
    
    capital_did = "Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;"
    capital_rows = md_meta[
        (md_meta[mc.did] == capital_did) &
        (md_meta[mc.stype] == "Seasonally Adjusted")
    ]
    
    if len(capital_rows) == 0:
        capital_rows = md_meta[
            md_meta[mc.did].str.contains("Net capital stock", case=False, na=False) &
            md_meta[mc.did].str.contains("Chain volume", case=False, na=False) &
            (md_meta[mc.stype] == "Seasonally Adjusted")
        ]
    
    if len(capital_rows) == 0:
        print("Could not find capital stock series")
        return
    
    capital_row = capital_rows.iloc[0]
    capital_id = capital_row[mc.id]
    capital_table = capital_row[mc.table]
    capital = md_dict[capital_table][capital_id].dropna()
    print(f"Using: {capital_row[mc.did]}")
    
    # --- Get Hours Worked (from Key Aggregates) ---
    hours_selector = {
        KEY_AGGS: mc.table,
        series_type: mc.stype,
        "Hours worked: Index ;": mc.did,
    }
    _table, hours_id, _units = ra.find_abs_id(meta, hours_selector, verbose=False)
    hours = abs_dict[KEY_AGGS][hours_id].dropna()
    
    # --- Normalize to index form ---
    Y_norm = gdp / gdp.iloc[0] * 100
    K_norm = capital / capital.iloc[0] * 100
    L_norm = hours / hours.iloc[0] * 100
    
    # --- Capital Productivity (Y/K) - pandas handles alignment ---
    capital_prod = (Y_norm / K_norm * 100).dropna()
    capital_prod.name = "Capital Productivity (Y/K)"
    
    common: dict[str, Any] = {
        "rfooter": f"{source} + ABS 1364.0.15.003",
        "lfooter": f"Australia. {series_type}. Output per unit of capital (indexed). ",
        "pre_tag": "capital-productivity-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }
    
    # Level charts
    multi_start(
        capital_prod,
        function=line_plot_finalise,
        starts=line_starts,
        title="Capital Productivity (Output per Unit of Capital)",
        ylabel=f"Index ({capital_prod.index[0]} = 100)",
        annotate=True,
        **common,
    )
    
    # Growth chart
    hma_term = 13
    cp_growth = capital_prod.pct_change(periods=4) * 100
    cp_growth = cp_growth.dropna()
    cp_growth_hma = hma(cp_growth, hma_term)
    
    growth_df = pd.DataFrame({
        "Annual growth": cp_growth,
        f"{hma_term}-term HMA": cp_growth_hma,
    })
    
    line_plot_finalise(
        growth_df,
        title="Capital Productivity Growth",
        ylabel="Per cent (YoY)",
        width=[1, 3],
        annotate=[False, True],
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth",
        **common,
    )
    
    # --- Capital-Output Ratio (K/Y) ---
    capital_output_ratio = (K_norm / Y_norm * 100).dropna()
    capital_output_ratio.name = "Capital-Output Ratio (K/Y)"
    
    common_ky: dict[str, Any] = {
        "rfooter": f"{source} + ABS 1364.0.15.003",
        "lfooter": f"Australia. {series_type}. Capital per unit of output (indexed). ",
        "pre_tag": "capital-output-ratio-",
        "show": SHOW,
        "file_type": FILE_TYPE,
    }
    
    multi_start(
        capital_output_ratio,
        function=line_plot_finalise,
        starts=line_starts,
        title="Capital-Output Ratio (Capital per Unit of Output)",
        ylabel=f"Index ({capital_output_ratio.index[0]} = 100)",
        annotate=True,
        **common_ky,
    )
    
    # --- Comparison: All three productivity measures ---
    labour_prod = (Y_norm / L_norm * 100).dropna()
    capital_deepening = (K_norm / L_norm * 100).dropna()
    
    comparison_df = pd.DataFrame({
        "Labour Productivity (Y/L)": labour_prod,
        "Capital Deepening (K/L)": capital_deepening,
        "Capital Productivity (Y/K)": capital_prod,
    }).dropna()
    
    line_plot_finalise(
        comparison_df,
        title="Productivity Measures Comparison",
        ylabel="Index (start = 100)",
        annotate=True,
        legend={"loc": "best", "fontsize": 9},
        tag="comparison",
        lfooter=f"Australia. {series_type}. Y/L = (K/L) × (Y/K). ",
        rfooter=f"{source} + ABS 1364.0.15.003",
        pre_tag="capital-productivity-",
        show=SHOW,
        file_type=FILE_TYPE,
    )
    
    # Growth comparison
    lp_growth = labour_prod.pct_change(periods=4) * 100
    kl_growth = capital_deepening.pct_change(periods=4) * 100
    
    growth_compare = pd.DataFrame({
        "Labour Productivity (Y/L)": lp_growth,
        "Capital Deepening (K/L)": kl_growth,
        "Capital Productivity (Y/K)": cp_growth,
    }).dropna()
    
    line_plot_finalise(
        growth_compare,
        title="Productivity Growth Comparison",
        ylabel="Per cent (YoY)",
        annotate=True,
        y0=True,
        legend={"loc": "best", "fontsize": 9},
        tag="growth-comparison",
        lfooter=f"Australia. {series_type}. ",
        rfooter=f"{source} + ABS 1364.0.15.003",
        pre_tag="capital-productivity-",
        show=SHOW,
        file_type=FILE_TYPE,
    )
    
    print(f"Capital productivity (latest): {capital_prod.iloc[-1]:.1f}")
    print(f"Capital productivity growth (latest YoY): {cp_growth.iloc[-1]:.2f}%")
    print(f"Capital-output ratio (latest): {capital_output_ratio.iloc[-1]:.1f}")


capital_productivity()

Fetching capital stock from ABS Modellers Database (1364.0.15.003)...
Using: Non-financial and financial corporations ; Net capital stock (Chain volume measures) ;
Capital productivity (latest): 42.4
Capital productivity growth (latest YoY): -0.17%
Capital-output ratio (latest): 236.0


### Real Wages (using implicit price deflator)

In [15]:
def real_wages(deflators: dict[str, pd.Series]) -> None:
    """Plot real wages."""

    table = "5206024_Selected_Analytical_Series"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    price_type = "Current Prices"
    dids = [
        "Average compensation per employee: Current prices ;",
        "Compensation of employees per hour: Current prices ;",
    ]

    for did in dids:
        deflator = "Households"
        row = meta[
            (meta[mc.table] == table)
            & (meta[mc.stype] == series_type)
            & (meta[mc.did] == did)
        ].iloc[0]
        series_id, units, did = row[mc.id], row[mc.unit], row[mc.did]
        series = (data[series_id] / deflators[deflator]).dropna()
        title = did.split(":")[0].strip()
        series.name = title

        suffix = "" if "per hour" in did else " / Qtr"
        common = {
            "title": f"Real {title}",
            "ylabel": f"{units} (inflation adjusted){suffix}",
            "rfooter": f"{source} {table}",
            "lfooter": f"Australia. {series_type.capitalize()} series. "
            f"{price_type.capitalize()} adjusted by {deflator} deflator. ",
            "pre_tag": "wages-",
            "show": SHOW,
            "file_type": FILE_TYPE,
        }

        line_plot_finalise(
            series,
            annotate=True,
            **common,
        )

        postcovid_plot_finalise(
            series,
            tag="covid",
            annotate=[False, True],
            **common,
        )


real_wages(DEFLATORS)

## Watermark

In [16]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2026-01-10 22:46:50

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.9.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

mgplot     : 0.2.18
numpy      : 2.4.0
pandas     : 2.3.3
readabs    : 0.1.8
statsmodels: 0.14.6
typing     : 3.10.0.0

Watermark: 2.6.0



In [17]:
print("Finished")

Finished
